In [61]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
import numpy
import numpy as np
import time
import ipyparallel as ipp
from qiskit import Aer
import copy
import qiskit
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit.quantum_info import Pauli
from qiskit.opflow import I,X,Y,Z
from qiskit.opflow.primitive_ops import PauliSumOp
from openfermion.linalg import givens_decomposition_square as givensRotns
from qiskit_nature.problems.second_quantization.electronic.builders import fermionic_op_builder
from qiskit_nature.mappers.second_quantization import JordanWignerMapper, ParityMapper
from qiskit import QuantumCircuit, QuantumRegister

In [62]:
def egBandHamiltonian(U):
    #Getting chemical Potential for Half-Filling
    with open('qcFinal/EmbeddingTwoBandHamiltonians/chem_pot_for_Half_Fill.txt','r') as f:
        lines=f.readlines()[1:]
        for line in lines:
            elems=line.split()
            if int(elems[0])==U:
                muHalf=float(elems[1]) #Chem Pot for a given Hubbard U
    #Getting the one body and two body interaction vertexes
    with open('qcFinal/EmbeddingTwoBandHamiltonians/'+str(U)+'/v1e.dat','r') as f:
            lines=f.readlines()[1:]
            num_sites=4
            chem_pot=numpy.zeros((2*num_sites,2*num_sites))
            eg_h1=numpy.zeros((2*num_sites,2*num_sites))
            for line in lines:
                elems=line.split()
                eg_h1[int(elems[0])][int(elems[1])]=float(elems[2])
                eg_h1[int(elems[0])+num_sites][int(elems[1])+num_sites]=float(elems[2])
            for i in range(2*num_sites):
                chem_pot[i][i]=-muHalf
            eg_h1=eg_h1+chem_pot       
    with open('qcFinal/EmbeddingTwoBandHamiltonians/'+str(U)+'/v2e.dat','r') as f:
        num_sites=4
        eg_h2=numpy.zeros((2*num_sites,2*num_sites,2*num_sites,2*num_sites))
        for line in f:
            if "#" in line:
                continue
            line = line.split()
            i,j,k,l = map(int, line[:4])
            val = float(line[4])
            eg_h2[i,j,k,l] = eg_h2[i+num_sites,j+num_sites,k,l] = eg_h2[i,j,k+num_sites,l+num_sites]             = eg_h2[i+num_sites,j+num_sites,k+num_sites,l+num_sites] = 0.5*val  # convention with 0.5 factor included.
    return eg_h1,eg_h2
def add_multiqubit_gate(pauli_string, param, circuit):
    num_qubits=circuit.num_qubits
    qr=QuantumRegister(num_qubits,'q')
    if pauli_string == 'I'*num_qubits:
        gate = 1
        for j in range(len(pauli_string)):
            gate = numpy.kron(gate, Pauli('I').to_matrix())
        gate *= -1j * np.sin(param)
        gate += numpy.cos(param) * np.eye(2**num_qubits)
        circuit.unitary(gate, qr, label=pauli_string)
    else:
        qubits_to_act_on = []
        gate = 1
        for j in range(len(pauli_string)):
            if pauli_string[j] != 'I':
                gate=numpy.kron(Pauli(pauli_string[j]).to_matrix(),gate)
                qubits_to_act_on.append(num_qubits-j-1)
        gate *= (-1j * numpy.sin(param))
        gate += numpy.cos(param) * numpy.eye(2**len(qubits_to_act_on))
        List_regs=[qr[i] for i in qubits_to_act_on][::-1]
        Label=pauli_string+"\n{:0.02f}".format(param.real)
        circuit.unitary(gate, List_regs, label = Label)
    return circuit
def qubitOp(h1,h2):
    num_qubits=h1.shape[0]
    fer_op=fermionic_op_builder.build_ferm_op_from_ints(h1,h2)
    mapper=ParityMapper()
    qubit_op=mapper.map(fer_op)
    qubit_conv = QubitConverter(mapper, two_qubit_reduction=True, z2symmetry_reduction="auto")
    qubit_op = qubit_conv.convert(fer_op, num_qubits//2)
    return qubit_op
def get_cholesky_vectors(h2):
    def reshape_vec_to_mat(eigs):
        weight,vec=eigs
        num_qubits=int(numpy.sqrt(vec.shape[0]))
        L=numpy.reshape(vec,(num_qubits,num_qubits))
        return [weight,L]
    num_qubits=h2.shape[0]
    N2=num_qubits*num_qubits
    h2_ps_qr=numpy.reshape(h2,(N2,N2))
    #Diagonalize Super matrix
    w,v=numpy.linalg.eigh(h2_ps_qr)
    #Build Cholesky vectors
    Larr=[]
    arr=[(w[i],v[:,i]) for i in range(len(v))]
    Larr=list(map(reshape_vec_to_mat,arr))
    Larr=[Larr[i] for i in range(len(Larr)) if numpy.abs(Larr[i][0])>1e-2]
    return Larr
def LRF_circuits(h):
        num_qubits=h.shape[0]
        qubitH=qubitOp(h,numpy.zeros((num_qubits,num_qubits,num_qubits,num_qubits)))
        num_qubits_P=qubitH.num_qubits
        H=qubitH.to_matrix()
        w,v=numpy.linalg.eigh(h)
        givens=givensRotns(v.T)[0]
        M_op=PauliSumOp.from_list([('I'*num_qubits,1)])
        rotH=qubitH
        circ=QuantumCircuit(num_qubits_P)
        for rots in givens:
            for tup in rots:
                i,j,theta,phi=tup
                h1=numpy.zeros((num_qubits,num_qubits))*1j
                h1[i,j]=-1j
                h1[j,i]=1j
                qubit_h1=qubitOp(h1,numpy.zeros((num_qubits,num_qubits,num_qubits,num_qubits)))
                List=qubit_h1.primitive.to_list()
                Op1=[('I'*num_qubits_P,numpy.cos(theta*List[0][1])),(List[0][0],1j*numpy.sin(theta*List[0][1]))]
                Op2=[('I'*num_qubits_P,numpy.cos(theta*List[1][1])),(List[1][0],1j*numpy.sin(theta*List[1][1]))]
                Op1=PauliSumOp.from_list(Op1)
                Op2=PauliSumOp.from_list(Op2)
                Op=Op2@Op1
                M_op=Op@M_op
                M_op=M_op.reduce().reduce()
                rotH=Op@rotH@Op.adjoint()
                rotH=rotH.reduce().reduce()
                circ=add_multiqubit_gate(List[0][0], theta*List[0][1], circ)
                circ=add_multiqubit_gate(List[1][0], theta*List[1][1], circ)
        return circ,rotH,qubitH

In [63]:
U=7
eg_h1,eg_h2=egBandHamiltonian(U)

In [64]:
#cholesky vectors for the four fermi term
Larr=get_cholesky_vectors(eg_h2)

In [65]:
#givens rotn for hopping terms
circ,rotH,qubitH_1=LRF_circuits(eg_h1)

In [66]:
#rotated Hamiltonian
print(rotH)

(-18.909397921999993+3.469446951953614e-17j) * IIIIII
- 0.166149803024346 * ZIIIII
+ (4.893499283524346-1.1092357608687972e-16j) * ZZIIII
+ (4.893499283524346+2.6963019221421275e-33j) * IIZIII
+ (-0.16614980302434645-8.085607530171071e-19j) * IZZIII
+ (-0.1661498030243464+2.696301922142131e-33j) * IIIZII
+ (-0.16614980302434634-1.1092357608687982e-16j) * IIIZZI
+ (4.893499283524349-1.8488927466117464e-32j) * IIIIIZ
+ (4.893499283524348+4.136552504734431e-19j) * IIIIZZ


In [67]:
#original Hamiltonian
print(qubitH)

-0.24999999999999994 * IIXIII
+ (0.24999999999999994-1.3877787807814457e-17j) * IZXIII
- 0.24999999999999992 * IIIIIX
+ 0.24999999999999992 * IIIIZX


In [58]:
#Cholesky vector rotation
circ,rotH,qubitH=LRF_circuits(Larr[1][1])